In [1]:
include("../src/Julia.jl")

pcgWorker (generic function with 1 method)

In [2]:
pwd()

"/Users/serbanstan/git/TreePCG/julia/testing"

In [3]:
graphp = "../../graphs/grid3_iu_27000_1/"

"../../graphs/grid3_iu_27000_1/"

In [4]:
a = readFromFile(graphp * "graph.mtx");

In [5]:
tree = []
for i in 1:3
    tr = readFromFile(graphp * "tree$(i).mtx");
    push!(tree, tr);
end

In [6]:
x = readFromFile(graphp * "x.vec");

In [7]:
la = lap(a);
x = x - mean(x)
b = la * x; b = b - mean(b);

In [8]:
numIts = 200;

@time for i in 1:3
    @time F = treeSolver(tree[i])
    @time F(b);
    
    # a run on standard Float64 data types
    @time myx64,dbg = pcgV(la, b, F, x, maxits=numIts, verbose=false);
    norm(lap(a) * myx64 - b) / norm(b)
    insert!(dbg, 1, "$(numIts) iterations")
    insert!(dbg, 1, "anorm = sqrt((x-truex)' * la * (x-truex)), err2_a = norm(la*x-b) / norm(b), err2_b = norm(x-truex)")
    writeToFile(graphp * "_log_julia_tree$(i)_default.txt", dbg)
    
    # now useing higer precision data types
    for hp in [64,128,256,512,1024]
        set_bigfloat_precision(hp)
        laHP = toHighPrecision(la; precision=hp)
        treeHP = toHighPrecision(tree[i]; precision=hp)
        xHP = toHighPrecision(x; precision=hp);
        bHP = toHighPrecision(b; precision=hp);
        FHP = treeSolver(treeHP)

        @time myxHP,dbg = pcgV(laHP, bHP, FHP, xHP, maxits=numIts, verbose=true);
        insert!(dbg, 1, "$(numIts) iterations")
        insert!(dbg, 1, "anorm = sqrt((x-truex)' * la * (x-truex)), err2_a = norm(la*x-b) / norm(b), err2_b = norm(x-truex)")
        writeToFile(graphp * "_log_julia_tree$(i)_$(hp).txt", dbg)
    end
end

  0.679442 seconds (535.44 k allocations: 29.665 MB, 2.60% gc time)
  0.341960 seconds (539.70 k allocations: 12.914 MB, 1.84% gc time)
  8.799303 seconds (86.76 M allocations: 2.193 GB, 4.40% gc time)
Working on iteration 10 with a-norm error 32.51676942201925
Working on iteration 20 with a-norm error 29.34440561156777
Working on iteration 30 with a-norm error 25.1070364948902
Working on iteration 40 with a-norm error 20.553821075799885
Working on iteration 50 with a-norm error 16.653210029152554
Working on iteration 60 with a-norm error 12.153530588158858
Working on iteration 70 with a-norm error 9.447370515196566
Working on iteration 80 with a-norm error 6.993994093748651
Working on iteration 90 with a-norm error 5.346374547049396
Working on iteration 100 with a-norm error 4.205367061481635
Working on iteration 110 with a-norm error 3.1769176069647354
Working on iteration 120 with a-norm error 2.379477071403941
Working on iteration 130 with a-norm error 1.933666579150614
Working on 